In [2]:
import sys
sys.path.append('../src')
from utils.satellite_image import SatelliteImage
from utils.utils import *
from utils.plot_utils import *

import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
#import cv2
from PIL import Image as im

from datetime import date
import re
import pyproj
from pyproj import Transformer


In [14]:
update_storage_access()
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]

bucket = environment["bucket"]

path_s3_pleiades_data_2017_mayotte = environment["sources"]["PLEIADES"][2017]["mayotte"]
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["mayotte"]
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["mayotte"]

path_local_pleiades_data_2017_mayotte = environment["local-path"]["PLEIADES"][2017]["mayotte"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["mayotte"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["mayotte"]


fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [15]:
# DL PLEIADE martinique 2017
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_mayotte}",
        lpath=f"../{path_local_pleiades_data_2017_mayotte}",
        recursive=True)

# DL PLEIADE martinique 2022
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2019_mayotte}",
        lpath=f"../{path_local_pleiades_data_2019_mayotte}",
        recursive=True)

fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_mayotte}",
        lpath=f"../{path_local_pleiades_data_2022_mayotte}",
        recursive=True)

In [39]:
str(3)

'3'

In [45]:

#crs = 5490 pour la martinique
def gps_to_crs_point(lat,lon,crs) :
    
    # coordinate  = (-12.789072624567547, 45.21788967618977)
    # lat, lon = coordinate
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:"+ str(crs))

    # Convertir les coordonnées GPS en coordonnées dans le système de coordonnées de destination (CRS)
    x, y = transformer.transform(lat, lon)
    
    # Retourner les coordonnées dans le CRS spécifié
    return x, y

#ca serait bien de faire une fonction qui cherche dans l'ensemble des données l'image qui contiendra cette coordonées.

def find_image_of_point(coordinate, folder_path, coord_gps = False, crs = None) :
        
    if coord_gps == True :  
        lat, lon = coordinate
        x,y = gps_to_crs_point(lat,lon,crs) #car y=lat et x=lon, les coordonnées gps sont en (lat,lon)
    else :
        x,y = coordinate
                                          
    for filename in os.listdir(folder_path):
        
        # Define a list of delimiter characters
        delimiters = ["-", "_"]

        # Create a regular expression pattern from the delimiter characters
        pattern = "|".join(delimiters)

        # Split the string using the regular expression pattern
        split_filename = re.split(pattern, filename)
        
        left = float(split_filename[2])*1000
        top = float(split_filename[3])*1000
        right = left + 1000.0
        bottom = top - 1000.0
         
        if left <= x <= right:
            if bottom <= y <= top:
                return(folder_path + '/' +filename)
    else : 
        return("Le point n'est pas retrouvé dans ce fichier d'images")
           


In [44]:
#len(os.listdir("../data/PLEIADES/2017/MAYOTTE"))
#len(os.listdir("../data/PLEIADES/2019/MAYOTTE"))
#len(os.listdir("../data/PLEIADES/2022/MAYOTTE"))


673

In [24]:
folder_path1 = "../data/PLEIADES/2017/MAYOTTE/"
folder_path2 = "../data/PLEIADES/2019/MAYOTTE/"
path_t = folder_path2 + os.listdir(folder_path2)[59]

image_t = SatelliteImage.from_raster(
        path_t,
        date = 2019, 
        n_bands = 3,
        dep = "972"
    )

#image_t.plot([0,1,2])
image_t.crs

CRS.from_epsg(4471)

In [ ]:
#
# 14.769096787469728, -60.90006133637827 -> badant nuages..
# 14.760965395697703, -60.9138387873933 -> idem..
coordinate  = (-12.789072624567547, 45.21788967618977)


folder_path1 = "../data/PLEIADES/2017/MAYOTTE"
folder_path2 = "../data/PLEIADES/2019/MAYOTTE"
coord_gps = True
path_t = find_image_of_point(
                                   coordinate,
                                   folder_path1,
                                   coord_gps,
                                    crs = 4471
                                  )
path_tplus1= find_image_of_point(
                                   coordinate,
                                   folder_path2,
                                   coord_gps,
                                crs = 4471
                                  )




#fromraster !!
#on recharge l'image originale
image_t = SatelliteImage.from_raster(
        path_t,
        date = 2017, 
        n_bands = 3,
        dep = "972"
    )
image_tplus1 = SatelliteImage.from_raster(
        path_tplus1,
        date = 2019, 
        n_bands = 3,
        dep = "972"
    )


image_t.plot([0,1,2])
image_tplus1.plot([0,1,2])